# Assignment 8 - Group 1

**Group members** :
- Max Chipani
- Jesus Gamboa
- Karen Salazar
- Paolo Gutierrez
- Luis Camarena

In [5]:
#The following command installs the geopandas, rasterio and rasterstats libraries.
!pip install geopandas rasterio rasterstats

In [6]:
# Import bookstores
import geopandas as gpd #Import the GeoPandas library to work with geospatial data in vector format (points, lines, polygons).
import pandas as pd # Import pandas for data manipulation in the form of tables (DataFrames).
import numpy as np # Import NumPy for numerical operations and array handling.
import matplotlib.pyplot as plt # Import Matplotlib to create graphs and visualizations.
import rasterio # Import raster to work with raster data (georeferenced images).
import pyproj # paraImport pyproj for geographic coordinate projection.
import glob # Import glob find files that match a specific pattern on the file system.
# Importa Folium para crear mapas interactivos basados en Leaflet.js.
import folium as fm # Import Folium to create interactive maps based on Leaflet.js.
from rasterstats import zonal_stats # Import the zonal_stats function from rasterstats to calculate zonal statistics between raster and vector data.
from shapely.ops import transform # Import shapely's transform function to transform geometries, such as changing the coordinate system.

## Question 1

Get the 15 variables from this raster for all Peru departments polygons. This is the link where shapefiles are located. This is the link of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.

In [11]:
# Read shapefile containing district boundaries 
dist = gpd.read_file("../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp")

# Select relevant columns
dist = dist[['CCDD', 'NOMBDEP', 'NOMBDIST', 'UBIGEO', 'geometry']]

# Display the resulting GeoDataFrame with the selected columns
dist

,CCDD,NOMBDEP,NOMBDIST,UBIGEO,geometry
0,06,CAJAMARCA,GUZMANGO,060504,"POLYGON ((-78.91578 -7.32085, -78.9137 -7.3223..."
1,06,CAJAMARCA,EL PRADO,061105,"POLYGON ((-79.01679 -6.96945, -79.01581 -6.969..."
2,06,CAJAMARCA,NIEPOS,061109,"POLYGON ((-79.21691 -6.85171, -79.21576 -6.851..."
3,06,CAJAMARCA,SAN GREGORIO,061110,"POLYGON ((-79.08233 -7.00544, -79.08041 -7.005..."
4,06,CAJAMARCA,SAN LUIS,061203,"POLYGON ((-78.87623 -7.09747, -78.87471 -7.098..."
...,...,...,...,...,...
1869,04,AREQUIPA,CHACHAS,040404,"POLYGON ((-71.95148 -14.83836, -71.95157 -14.8..."
1870,04,AREQUIPA,CHILCAYMARCA,040405,"POLYGON ((-72.3749 -15.2499, -72.37494 -15.250..."
1871,04,AREQUIPA,CHOCO,040406,"POLYGON ((-71.96976 -15.13597, -71.96952 -15.1..."
1872,04,AREQUIPA,HUANCARQUI,040407,"POLYGON ((-72.31258 -15.81426, -72.31211 -15.8..."


In [ ]:
# Set up a transformer to convert coordinates from WGS84 to Mollweide projection
transformer = pyproj.Transformer.from_crs('epsg:4326', 'esri:54009', always_xy=True)

# Define a function to apply the coordinate transformation to geometries
def apply_transform(geom):
    return transform(transformer.transform, geom)

# Apply the transformation to the districts geometries
dist['geometry'] = dist['geometry'].apply(apply_transform)

In [ ]:
# Get a list of all .tif raster files in the current directory
lista_rasters = glob.glob("*.tif")
lista_rasters

In [ ]:
x = []

# Obtain the values counts in each raster with district geometry
for raster in lista_rasters:
    print(raster)
    stats = zonal_stats(dist, raster,
                       stats='unique',
                       categorical=True)
    df1 = pd.DataFrame(stats)
    df1['UBIGEO'] = dist['UBIGEO']
    x.append(df1)

In [ ]:
# Create the value counts of each variable at district level
df_ubigeo = pd.concat(x).replace(np.nan, 0).groupby('UBIGEO').sum().reset_index()
df_ubigeo

In [ ]:
# Create the value counts of each variable at departament level
df_final = pd.merge(dist, df_ubigeo, on='UBIGEO',  how='left')
df_final = df_final.drop(['UBIGEO', 'geometry'], axis=1)
df_final = df_final.groupby(['CCDD', 'NOMBDEP']).sum()
df_final = df_final.reset_index()
df_final = df_final.drop(['NOMBDIST', 'unique', 0], axis=1)
df_final

In [ ]:
# Calculate the percentage of district area cover by this specific Morphological Settlement Zone.
variables = df_final.columns[2:]
for var in variables:
    df_final[f'MSZ_{var}'] = df_final[var] / df_final[var].sum() * 100

df_final = df_final[['CCDD', 'NOMBDEP'] + [f'MSZ_{var}' for var in variables]]
df_final = df_final.replace(np.nan, 0)
df_final

## Question 2

Then you are going to generate choropleth map using folium for these 15 variables.